# Llama Document Summary Index With Amazon Bedrock Claude Model
> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


This demo showcases the document summary index.

The document summary index will extract a summary from each document and store that summary, as well as all nodes corresponding to the document.

Retrieval can be performed through the LLM or embeddings. We first select the relevant documents to the query based on their summaries. All retrieved nodes corresponding to the selected documents are retrieved.

This notebook demonstrates invoking Bedrock models directly using the AWS SDK, but for later notebooks in the workshop you'll also need to install [LangChain](https://github.com/hwchase17/langchain):

In [2]:
%pip install langchain==0.0.309 --force-reinstall --quiet
%pip install pypdf==3.15.2 --force-reinstall --quiet
%pip install llama-index==0.8.37 --force-reinstall --quiet
%pip install sentence_transformers==2.2.2 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
hdijupyterutils 0.20.5 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.0.6 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,

In [3]:
%pip install pydantic==1.10.13 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.2.2 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.15.0 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.0a7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nest_asyncio

nest_asyncio.apply()

In [6]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    get_response_synthesizer,
    set_global_service_context
)
from llama_index.indices.document_summary import DocumentSummaryIndex

### Load Datasets

In [7]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [8]:
import glob
from pypdf import PdfReader, PdfWriter

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()


Now that you have clean PDFs to work with, you will enrich your documents with metadata, then use a process called "chunking" to break up a larger document into small pieces. These small pieces will allow you to generate embeddings without surpassing the input limit of the embedding model.

In this example you will break the document into 1000 character chunks, with a 100 character overlap. This will allow your embeddings to maintain some of its context.

In [9]:
docs = []
for filename in filenames:
    doc = SimpleDirectoryReader(input_files=[f"data/{filename}"]).load_data()
    doc[0].doc_id = filename.replace(".pdf", "")
    docs.extend(doc)

### Build Document Summary Index

We show two ways of building the index:
- default mode of building the document summary index
- customizing the summary query


In [10]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = 'bedrock-user'

In [11]:
import os
import boto3
import json
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

bedrock_client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True # Default. Needed for invoke_model() from the data plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [12]:
from llama_index import LangchainEmbedding
from langchain.llms.bedrock import Bedrock # required until llama_index offers direct Bedrock integration
from langchain.embeddings.bedrock import BedrockEmbeddings

model_kwargs_claude = {
    "temperature": 0,
    "top_k": 10,
    "max_tokens_to_sample": 512
}

llm = Bedrock(model_id="anthropic.claude-v2",
              model_kwargs=model_kwargs_claude)

embed_model = LangchainEmbedding(
  BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")
)

service_context = ServiceContext.from_defaults(llm=llm, 
                                               embed_model=embed_model, 
                                               chunk_size=512)
set_global_service_context(service_context)

### This will take a few minutes. Please be patient.

In [13]:
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    docs,
    service_context=service_context,
    response_synthesizer=response_synthesizer,
)

current doc id: AMZN-2022-Shareholder-Letter
current doc id: 51a0ae06-4d93-438d-95a1-ae325ecbe17e
current doc id: 7e9fa3f5-9a63-4d28-a736-f38d7955004c
current doc id: 7ba25de4-892f-48c2-96bd-0dc643d0aa8f
current doc id: 5b8adc75-5575-44d4-9db9-dc237aeea004
current doc id: c5774527-5fa1-4570-94df-17090350792b
current doc id: 0d225840-8ffc-472d-a41f-997bc90f3877
current doc id: AMZN-2021-Shareholder-Letter
current doc id: b2155879-ea7f-4ffe-a17a-ccc325e7833a
current doc id: aad38bbd-2904-4f87-ad67-d4b59d2e957e
current doc id: d681ed2f-6d5b-4a6e-8969-fdc4309ab495
current doc id: bcbfaf14-af9e-4e08-9924-6b1633adea5e
current doc id: 5b1bab92-62d5-49e9-974a-bda00062a37f
current doc id: AMZN-2020-Shareholder-Letter
current doc id: 16e030f2-a57b-4147-ae99-cce254baebb1
current doc id: 28e5584e-d15a-46fa-b1a4-f694f8c9e72e
current doc id: 157a1ee3-b2cb-492d-88e1-e688e9c03356
current doc id: 4796ad3f-138e-4df6-9f96-4c675269c28d
current doc id: b3888a74-2581-488a-ae55-c8e93bf53a21
current doc id: 0

In [14]:
print(len(docs))

25


In [15]:
doc_summary_index.get_document_summary("AMZN-2022-Shareholder-Letter")

" Based on the context, this text is the opening letter from Amazon's 2022 annual shareholder letter written by CEO Andy Jassy. It provides an overview of Amazon's performance and outlook.\n\nSome key points and potential questions this text can answer:\n\n- It discusses Amazon's performance and challenges in 2022, a difficult macroeconomic year, but expresses optimism about the future. Questions it can answer - How did Amazon perform financially in 2022? What were some of the challenges it faced?\n\n- It talks about how Amazon innovated and made adjustments to investments and inventing. Questions - What innovations did Amazon make in 2022? How did it adjust its investment approach? \n\n- It reflects on how Amazon has constantly evolved over 25 years across different businesses like marketplace, AWS, Kindle. Questions - How has Amazon transformed as a business over time? How have its major business segments like retail, cloud computing evolved?\n\n- It provides examples of how Amazon h

In [16]:
doc_summary_index.storage_context.persist("llama_claude_index")

In [17]:
from llama_index.indices.loading import load_index_from_storage
from llama_index import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="llama_claude_index")
doc_summary_index = load_index_from_storage(storage_context)

### Perform Retrieval from Document Summary Index

We show how to execute queries at a high-level. We also show how to perform retrieval at a lower-level so that you can view the parameters that are in place. We show both LLM-based retrieval and embedding-based retrieval using the document summaries.

In [18]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompt_type import PromptType

CLAUDE_CHOICE_SELECT_PROMPT_TMPL = (
    """
    
    Human: A list of documents is shown below. Each document has a number next to it along  with a summary of the document. A question is also provided. 
    Respond with the respective document number. You should consult to answer the question, in order of relevance, as well as the relevance score. 
    The relevance score is a number from 1-10 based on how relevant you think the document is to the question.\n"
    Do not include any documents that are not relevant to the question. 
    
    Example format: 
    Document 1:\n<summary of document 1>
    Document 2:\n<summary of document 2>
    ...\n\n
    Document 10:\n<summary of document 10>
    
    Question: <question>
    Answer:
    Doc: 9, Relevance: 7
    Doc: 3, Relevance: 4
    Doc: 7, Relevance: 3

    Let's try this now: 
    {context_str}

    Question: {query_str}
    
    Assistant: Answer:"""
)
claude_choice_select_prompt = Prompt(
    CLAUDE_CHOICE_SELECT_PROMPT_TMPL, prompt_type=PromptType.CHOICE_SELECT
)

#### High-level Querying

Note: this uses the default, LLM-based form of retrieval

In [19]:
from util.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn
query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True, similarity_top_k=10,  verbose=False,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    choice_select_prompt=claude_choice_select_prompt
)


ModuleNotFoundError: No module named 'util'

In [ ]:
response = query_engine.query("How has AWS evolved?")
print(response)

#### LLM-based Retrieval

In [ ]:
from llama_index.indices.document_summary import DocumentSummaryIndexRetriever

In [ ]:
from utils.llama_custom_parse_choice_select_answer_fn import custom_parse_choice_select_answer_fn
retriever = DocumentSummaryIndexRetriever(
    doc_summary_index,
    choice_select_prompt=claude_choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    # service_context=service_context
)

In [ ]:
retrieved_nodes = retriever.retrieve("According to the documents, How has AWS evolved?")
print(len(retrieved_nodes))

In [ ]:
for i, node in enumerate(retrieved_nodes):
    print(node.score)
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")

In [ ]:
%%time
# use retriever as part of a query engine
from llama_index.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(response_mode="refine", use_async=True)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("How has AWS evolved?")
print(response)

#### Embedding-based Retrieval

In [ ]:
from llama_index.indices.document_summary import DocumentSummaryIndexEmbeddingRetriever

In [ ]:
retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    choice_select_prompt=claude_choice_select_prompt,
    # choice_batch_size=choice_batch_size,
    # format_node_batch_fn=format_node_batch_fn,
    parse_choice_select_answer_fn=custom_parse_choice_select_answer_fn,
    #service_context=service_context
)

In [ ]:
retrieved_nodes = retriever.retrieve("Human: How has AWS evolved?")

In [ ]:
len(retrieved_nodes)

In [ ]:
for i, node in enumerate(retrieved_nodes):
    
    print(node.node.get_text())
    print("-----------------------------------------------------------------------------------------------------------------------------------")
    